In [ ]:
import os
import math
from termcolor import colored
import numpy as np
import matplotlib
matplotlib.use('Agg')
import pandas as pd
#from data_helpers import load_data
from keras import callbacks
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
#from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from keras import backend as K
from keras.optimizers import Adam

from keras.layers import Embedding
import string
import numpy as np
import pandas as pd
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import concatenate
from keras.layers import Dropout
import random

import regex
from keras.layers.convolutional import Convolution1D, MaxPooling1D, AveragePooling1D

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Necessary imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

from functools import partial
import tensorflow as tf

import tensorflow as tf
from tensorflow.python.ops import nn
from tensorflow.python.keras import activations, regularizers, initializers, constraints, engine
from tensorflow.python.keras.utils import conv_utils
from tensorflow.python.keras.layers import Layer, deserialize, Conv1D
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Flatten, Dense,AveragePooling1D,GRU,Convolution1D, MaxPooling1D, AveragePooling1D,Embedding,Input, Dense, Dropout, Flatten

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

sns.set_style("whitegrid")
np.random.seed(0)


import nltk
!nltk.download('stopwords')



MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])


/bin/bash: -c: line 0: syntax error near unexpected token `'stopwords''
/bin/bash: -c: line 0: `nltk.download('stopwords')'


In [ ]:
!unzip /content/drive/MyDrive/wiki.simple.vec.zip

Archive:  /content/drive/MyDrive/wiki.simple.vec.zip
replace wiki.simple.vec? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: wiki.simple.vec         


In [ ]:
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('wiki.simple.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

1507it [00:00, 7647.74it/s]

loading word embeddings...


111052it [00:14, 7655.67it/s]

found 111052 word vectors


In [ ]:
dataset=pd.read_csv('Abstract_Author.csv')


k=0
X_train=[]
y_train=[]
n=155 #set n equal number of authors present in the dataset
p=40 #set p equal to number of tweets per user you want in training set
z=50 #Z is the total number of tweets per user
for i in range(0,n):  
  for j in range(k,k+p):
    X_train.append(dataset.iloc[j,0])
    y_train.append(dataset.iloc[j,1])
  k+=z

a=10 #set a equal to number of tweets per user you want in testing set
k=p
X_test=[]
y_test=[]
for i in range(0,n):
  for j in range(k,k+a):
    X_test.append(dataset.iloc[j,0])
    y_test.append(dataset.iloc[j,1])
  k+=z

X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)
X_train=X_train.iloc[:,:].values
X_test=X_test.iloc[:,:].values
y_train=y_train.iloc[:,:].values
y_test=y_test.iloc[:,:].values

train=np.concatenate((X_train,y_train),axis=1)
test=np.concatenate((X_test,y_test),axis=1)

np.random.shuffle(train)
np.random.shuffle(test)

train=pd.DataFrame(train)
test=pd.DataFrame(test)

X_train1=train.iloc[:,0]
X_test=test.iloc[:,0]
y_train=train.iloc[:,1]
y_test=test.iloc[:,1]


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_Y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_eval, y_train, y_eval = train_test_split(X_train1,y_train, test_size = 0.05, random_state = 0)

In [ ]:
print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(X_train1)  #leaky
word_seq_train = tokenizer.texts_to_sequences(X_train)
word_seq_test = tokenizer.texts_to_sequences(X_test)
word_seq_eval=tokenizer.texts_to_sequences(X_eval)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

tokenizing input data...
dictionary size:  20554


In [ ]:
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=100)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=100)
word_seq_eval = sequence.pad_sequences(word_seq_eval, maxlen=100)

In [ ]:
embed_dim = 300 

In [ ]:
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))+1
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 8547


In [ ]:
def create_vocab_set2():
    alphabet = (list(string.ascii_lowercase) + list(string.digits) +
                list(string.punctuation) + ['\n'] + [' '] )
    
    print('alphabet', alphabet)
    #x = input(print('enter'))
    alphabets = []
    for i in range(0, len(alphabet)):
        for j in range(0, len(alphabet)):
              alphabets.append(alphabet[i]+alphabet[j])
    #print(alphabets)
    #x = input(print('enter'))


    vocab_size = len(alphabets)
    check = set(alphabets)

    vocab = {}
    reverse_vocab = {}
    for ix, t in enumerate(alphabets):
        vocab[t] = ix
        reverse_vocab[ix] = t

    return vocab, reverse_vocab, vocab_size, check


In [ ]:
def create_vocab_set():
    alphabet = (list(string.ascii_lowercase) + list(string.digits) +
                list(string.punctuation) + ['\n'] + [' '] )
    vocab_size = len(alphabet)
    check = set(alphabet)

    vocab = {}
    reverse_vocab = {}
    for ix, t in enumerate(alphabet):
        vocab[t] = ix
        reverse_vocab[ix] = t

    return vocab, reverse_vocab, vocab_size, check

In [ ]:
maxlen = 100
vocab, reverse_vocab, vocab_size, check =create_vocab_set()

In [ ]:
def encode_data2(x, maxlen, vocab, vocab_size, check):              # For character embedding use this function instead of encode_data

    input_data = np.zeros((len(x), maxlen))
    for dix, sent in enumerate(x):
        counter = 0
        chars = list(sent.lower().replace(" ", ""))
        chars2 = []
        for i in range(len(chars)-1):
            chars2.append(chars[i] + chars[i+1])
        for i,c in enumerate(chars2):
            if counter >= maxlen:
                pass
            else:
                if c in check:
                    counter += 1
                    ix = vocab[c]
                    input_data[dix,counter-1] = ix

    return input_data 

In [ ]:
def encode_data(x, maxlen, vocab, vocab_size, check):

    input_data = np.zeros((len(x), maxlen, vocab_size))
    for dix, sent in enumerate(x):
        counter = 0
        sent_array = np.zeros((maxlen, vocab_size))
        chars = list(sent.lower())
        for c in chars:
            if counter >= maxlen:
                pass
            else:
                char_array = np.zeros(vocab_size, dtype=np.int)
                if c in check:
                    ix = vocab[c]
                    char_array[ix] = 1
                sent_array[counter, :] = char_array
                counter += 1
        input_data[dix, :, :] = sent_array

    return input_data


In [ ]:
char_seq_train = encode_data2(X_train, maxlen, vocab, vocab_size, check)
char_seq_test = encode_data2(X_test, maxlen, vocab, vocab_size, check)
char_seq_eval = encode_data2(X_eval, maxlen, vocab, vocab_size, check)

In [ ]:
char_seq_train.shape

(5890, 100)

In [ ]:
def model(filter_kernels, dense_outputs, maxlen, vocab_size, nb_filter,cat_output,nb_words,embed_dim):                                                  # For Character Embedding use this model instead of above model
    d = 300                                                             #Embedding Size
    Embedding_layer  = Embedding(vocab_size+1, d, input_length=maxlen)
    input1 = Input(shape=(maxlen,), name='input', dtype='float32')
    embed = Embedding_layer(input1)
    conv = Convolution1D(filters=nb_filter , kernel_size=filter_kernels[0], activation='relu',input_shape=(maxlen, d))(embed)
    conv = MaxPooling1D(pool_size=3)(conv)
    conv1 = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[1],activation='relu')(conv)
    conv1 = MaxPooling1D(pool_size=3)(conv1)
    conv2 = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[2],activation='relu')(conv1)
    conv2 = MaxPooling1D(pool_size=3)(conv2)
    character = Flatten()(conv2)

    input2 = Input(shape=(100,))
    embed2=Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=140, trainable=False)(input2)
    conv3 = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[0], activation='relu')(embed2)
    conv3 = MaxPooling1D(pool_size=3)(conv3)
    conv4 = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[1],activation='relu')(conv3)
    conv4 = MaxPooling1D(pool_size=3)(conv4)
    conv5 = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[2],activation='relu')(conv4)
    conv5 = MaxPooling1D(pool_size=3)(conv5)
    word = Flatten()(conv5)
    
    
    merged = concatenate([character, word])


    pred = Dense(155, activation='softmax', name='output')(merged)

    model = Model(inputs=[input1,input2] , outputs=pred)

    #sgd = SGD(lr=0.001, momentum=0.9)
    #model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

In [ ]:
from tensorflow.keras.layers import concatenate

In [ ]:
nb_filter = 450
dense_outputs = 256
filter_kernels = [3,4,5]
cat_output = 50
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model = model(filter_kernels, dense_outputs,maxlen, vocab_size, nb_filter, cat_output,nb_words,embed_dim)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 100, 300)     21300       input[0][0]                      
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 100, 300)     6166500     input_7[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
import tensorflow as tf

from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3) 
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


In [ ]:
model.fit([np.array(char_seq_train),np.array(word_seq_train)] , np.array(y_train),validation_data=([np.array(char_seq_eval),np.array(word_seq_eval)] , np.array(y_eval)),batch_size=16,epochs=5, verbose=1,callbacks=[callback])

In [ ]:
y_pred = model.predict([np.array(char_seq_test),np.array(word_seq_test)])

In [ ]:
y_pred=pd.DataFrame(y_pred)
y_pred=y_pred.eq(y_pred.where(y_pred != 0).max(1), axis=0).astype(int)
y_pred=y_pred.iloc[:,:].values

y_test=pd.DataFrame(y_test)
y_test=y_test.eq(y_test.where(y_test != 0).max(1), axis=0).astype(int)
y_test=y_test.iloc[:,:].values

result=[]
for i in range(0,len(y_test)):
  for j in range(0,len(y_test[0])):
    if(y_test[i][j]==1):
      result.append(j)

predicted=[]
for i in range(0,len(y_pred)):
  for j in range(0,len(y_pred[0])):
    if(y_pred[i][j]==1):
      predicted.append(j)

print(result)
print(predicted)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(result,predicted)

from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(result, predicted)) 
print('Report : ')
print(classification_report(result, predicted))

[19, 27, 10, 3, 23, 24, 28, 44, 11, 33, 49, 40, 30, 13, 28, 1, 22, 20, 2, 20, 13, 20, 49, 6, 38, 3, 16, 23, 14, 16, 27, 31, 6, 48, 39, 16, 13, 9, 33, 26, 1, 38, 12, 49, 30, 21, 49, 18, 39, 19, 31, 35, 46, 29, 0, 34, 28, 0, 42, 33, 33, 42, 43, 5, 31, 44, 1, 43, 47, 12, 6, 14, 20, 3, 41, 25, 45, 34, 46, 8, 6, 35, 24, 10, 13, 29, 49, 29, 7, 19, 2, 15, 15, 24, 14, 25, 4, 13, 4, 26, 37, 7, 5, 38, 27, 47, 7, 30, 40, 5, 45, 16, 41, 17, 21, 32, 22, 36, 35, 25, 9, 19, 22, 41, 45, 4, 26, 0, 43, 2, 11, 11, 1, 8, 37, 39, 28, 12, 41, 24, 22, 47, 5, 48, 10, 3, 14, 42, 10, 47, 17, 47, 30, 12, 4, 36, 22, 45, 29, 46, 27, 14, 32, 30, 36, 43, 11, 24, 40, 23, 38, 2, 18, 36, 32, 20, 25, 43, 3, 25, 7, 32, 15, 12, 17, 26, 17, 48, 9, 4, 8, 21, 37, 34, 1, 0, 15, 18, 31, 34, 44, 5, 8, 26, 23, 41, 35, 34, 9, 46, 35, 45, 17, 44, 18, 6, 16, 2, 29, 38, 23, 31, 10, 40, 7, 40, 39, 27, 39, 15, 19, 18, 48, 44, 37, 42, 48, 11, 21, 37, 8, 42, 32, 33, 28, 0, 21, 9, 46, 36]
[19, 27, 25, 3, 23, 40, 28, 44, 2, 33, 49, 40, 30